# Bayesian Logistic Regression (Variational Inference: Mean-Field Gaussian)

Variational inference approximates the posterior $p(\beta|D)$ with a tractable
distribution $q_\lambda(\beta)$ by maximizing the ELBO:

$$\mathcal{L}(\lambda) = \mathbb{E}_{q_\lambda}[\log p(D,\beta) - \log q_\lambda(\beta)]$$

We use a diagonal Gaussian $q(\beta)=\mathcal{N}(\mu, \text{diag}(\sigma^2))$
and optimize using stochastic gradients (reparameterization trick).


# Chapter 10 Bayesian ML (Predictive): Bayesian Logistic Regression (VI)

These notebooks mirror the *methods* highlighted in
`ml_finance_thoery/machine-learning-for-trading/10_bayesian_machine_learning/README.md`
and apply them to the local `dataset/cleaned/` asset universe to produce
out-of-sample predictions and a backtest using the same **vectorized** engine
used by the `notebooks/ML_Linear_Models_*` notebooks.


In [1]:
from __future__ import annotations

import math
from pathlib import Path
import sys

import numpy as np
import pandas as pd

SEED = 42
rng = np.random.default_rng(SEED)

def find_project_root(start: Path) -> Path:
    p = start.resolve()
    for _ in range(10):
        if (p / 'src').exists() and (p / 'dataset').exists():
            return p
        p = p.parent
    raise RuntimeError(f'Could not find project root from: {start!s}')

PROJECT_ROOT = find_project_root(Path.cwd())
CLEANED_DIR = PROJECT_ROOT / 'dataset' / 'cleaned'

# Ensure `src/` is on sys.path so `backtester` is importable
src_dir = PROJECT_ROOT / 'src'
if str(src_dir) not in sys.path:
    sys.path.append(str(src_dir))


In [2]:
from scipy.special import expit
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

ASSET_LIMIT = 50
N_TRAIN_SAMPLES = 80_000
N_STEPS = 1500
BATCH = 4096
LR = 1e-2
TAU2 = 10.0**2

def compute_features_fast(ohlcv: pd.DataFrame) -> pd.DataFrame:
    df = ohlcv.copy().sort_index()
    c = df['Close'].astype(float)
    h = df['High'].astype(float)
    l = df['Low'].astype(float)
    v = df['Volume'].astype(float)
    ret_1d = c.pct_change()
    hl_range_pct = (h - l) / c.replace(0.0, np.nan)
    vol_z_20 = (v - v.rolling(20).mean()) / v.rolling(20).std()
    out = pd.DataFrame({'ret_1d': ret_1d, 'hl_range_pct': hl_range_pct, 'vol_z_20': vol_z_20}).sort_index()
    out['ret_1d_fwd'] = out['ret_1d'].shift(-1)
    out['y_up_fwd'] = (out['ret_1d_fwd'] > 0).astype(int)
    return out

from backtester.data import load_cleaned_assets
files = sorted([p for p in CLEANED_DIR.glob('Asset_*.csv')])
symbols = [p.stem for p in files][:ASSET_LIMIT]
assets_ohlcv = load_cleaned_assets(symbols=symbols, cleaned_dir=str(CLEANED_DIR))

frames = []
for sym, df in assets_ohlcv.items():
    feat = compute_features_fast(df)
    feat['Asset_ID'] = sym
    frames.append(feat)
data = pd.concat(frames, axis=0).sort_index().dropna(subset=['y_up_fwd'])

# Time split
TRAIN_YEARS = 7
VAL_MONTHS = 18
TEST_MONTHS = 18

def align_to_trading_date(index: pd.DatetimeIndex, ts: pd.Timestamp) -> pd.Timestamp:
    pos = int(index.searchsorted(ts, side='left'))
    if pos >= len(index):
        return pd.Timestamp(index[-1])
    return pd.Timestamp(index[pos])

idx = pd.DatetimeIndex(data.index.unique()).sort_values()
end = pd.Timestamp(idx[-1])
raw_test_start = end - pd.DateOffset(months=TEST_MONTHS)
raw_val_start = raw_test_start - pd.DateOffset(months=VAL_MONTHS)
raw_train_start = raw_val_start - pd.DateOffset(years=TRAIN_YEARS)
test_start = align_to_trading_date(idx, pd.Timestamp(raw_test_start))
val_start = align_to_trading_date(idx, pd.Timestamp(raw_val_start))
train_start = align_to_trading_date(idx, pd.Timestamp(raw_train_start))

df_train = data.loc[(data.index >= train_start) & (data.index < val_start)].copy()
df_test = data.loc[(data.index >= test_start) & (data.index <= end)].copy()

feature_cols = [c for c in df_train.columns if c not in {'Asset_ID', 'ret_1d_fwd', 'y_up_fwd'}]
X_train = df_train[feature_cols].replace([np.inf, -np.inf], np.nan)
y_train = df_train['y_up_fwd'].astype(int).to_numpy()
X_test = df_test[feature_cols].replace([np.inf, -np.inf], np.nan)

imp = SimpleImputer(strategy='median')
scaler = StandardScaler()
Xtr = scaler.fit_transform(imp.fit_transform(X_train))
Xte = scaler.transform(imp.transform(X_test))
Xtr = np.hstack([np.ones((Xtr.shape[0], 1)), Xtr])
Xte = np.hstack([np.ones((Xte.shape[0], 1)), Xte])

# Subsample for speed
if Xtr.shape[0] > N_TRAIN_SAMPLES:
    idx_s = rng.choice(Xtr.shape[0], size=N_TRAIN_SAMPLES, replace=False)
    Xtr = Xtr[idx_s]
    y_train = y_train[idx_s]

d = Xtr.shape[1]

# Adam optimizer
mu = np.zeros(d)
log_var = np.full(d, -2.0)
m_mu = np.zeros(d); v_mu = np.zeros(d)
m_lv = np.zeros(d); v_lv = np.zeros(d)
b1 = 0.9; b2 = 0.999; eps = 1e-8

def elbo_grad(mu: np.ndarray, log_var: np.ndarray, Xb: np.ndarray, yb: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    # Single-sample Monte Carlo gradient via reparameterization
    std = np.exp(0.5 * log_var)
    z = rng.normal(size=mu.shape)
    beta = mu + std * z
    logits = Xb @ beta
    p = expit(logits)
    # loglik grad wrt beta
    g_beta = Xb.T @ (yb - p)
    # prior grad (no prior on intercept)
    g_beta[1:] += -(beta[1:] / TAU2)
    # q entropy term: grad of -log q = + (beta-mu)/var for mu, etc; reparam handles it
    # Gradient of ELBO wrt mu is g_beta
    g_mu = g_beta
    # Gradient wrt log_var: chain through std and beta = mu + std*z
    g_std = g_beta * z
    g_log_var = 0.5 * std * g_std + 0.5  # +0.5 from entropy of diagonal Gaussian
    g_log_var[0] = 0.5 * std[0] * g_std[0] + 0.5
    return g_mu, g_log_var

# Optimize
n = Xtr.shape[0]
for t in range(1, N_STEPS + 1):
    idx_b = rng.integers(0, n, size=BATCH)
    Xb = Xtr[idx_b]
    yb = y_train[idx_b]
    g_mu, g_lv = elbo_grad(mu, log_var, Xb, yb)
    # Adam updates
    m_mu = b1 * m_mu + (1 - b1) * g_mu
    v_mu = b2 * v_mu + (1 - b2) * (g_mu * g_mu)
    m_lv = b1 * m_lv + (1 - b1) * g_lv
    v_lv = b2 * v_lv + (1 - b2) * (g_lv * g_lv)
    m_mu_hat = m_mu / (1 - b1 ** t)
    v_mu_hat = v_mu / (1 - b2 ** t)
    m_lv_hat = m_lv / (1 - b1 ** t)
    v_lv_hat = v_lv / (1 - b2 ** t)
    mu += LR * m_mu_hat / (np.sqrt(v_mu_hat) + eps)
    log_var += LR * m_lv_hat / (np.sqrt(v_lv_hat) + eps)
    if t % 300 == 0:
        print('step', t, 'mean std(beta):', float(np.mean(np.exp(0.5 * log_var))))

# Predict by sampling from q
S = 200
std = np.exp(0.5 * log_var)
B = mu + std * rng.normal(size=(S, d))
p_samps = expit(Xte @ B.T)
p_mean = p_samps.mean(axis=1)

pred_long = pd.DataFrame({'Date': df_test.index, 'Asset_ID': df_test['Asset_ID'].to_numpy(), 'signal': p_mean - 0.5})
pred_matrix = pred_long.pivot_table(index='Date', columns='Asset_ID', values='signal', aggfunc='mean').sort_index()


step 300 mean std(beta): 0.1964275094119226
step 600 mean std(beta): 0.13588994822423545
step 900 mean std(beta): 0.1027306574298679
step 1200 mean std(beta): 0.08443441441206717
step 1500 mean std(beta): 0.07211911040625275


In [3]:
from IPython.display import display
from bokeh.io import output_notebook, show

from backtester.data import load_cleaned_assets, align_close_prices
from backtester.engine import BacktestConfig, run_backtest
from backtester.report import compute_backtest_report
from backtester.bokeh_plots import build_interactive_portfolio_layout
from backtester.portfolio import equal_weight, optimize_mpt

output_notebook()

if 'pred_matrix' not in globals():
    raise RuntimeError('Expected `pred_matrix` (index=date, columns=Asset_ID) to exist')

# IMPORTANT: performance metrics should start when the model has signals.
# Keep the original prediction date range before reindexing to market data.
pred_range = pd.DatetimeIndex(pred_matrix.index).sort_values()
if pred_range.empty:
    raise RuntimeError('pred_matrix has empty index')
bt_start = pd.Timestamp(pred_range[0])
bt_end = pd.Timestamp(pred_range[-1])

bt_assets = sorted([str(c) for c in pred_matrix.columns.tolist()])
assets_ohlcv = load_cleaned_assets(symbols=bt_assets, cleaned_dir=str(CLEANED_DIR))
close_prices = align_close_prices(assets_ohlcv)

# Align prediction matrix to available trade dates
pred_matrix = pred_matrix.reindex(close_prices.index)
# Restrict backtest to the prediction window (avoid 2016-start metrics).
close_prices = close_prices.loc[bt_start:bt_end]
pred_matrix = pred_matrix.loc[bt_start:bt_end]
returns_matrix = close_prices.pct_change().fillna(0.0)

market_df = pd.DataFrame({
    'Open': pd.concat([df['Open'] for df in assets_ohlcv.values()], axis=1).mean(axis=1),
    'High': pd.concat([df['High'] for df in assets_ohlcv.values()], axis=1).mean(axis=1),
    'Low': pd.concat([df['Low'] for df in assets_ohlcv.values()], axis=1).mean(axis=1),
    'Close': pd.concat([df['Close'] for df in assets_ohlcv.values()], axis=1).mean(axis=1),
    'Volume': pd.concat([df['Volume'] for df in assets_ohlcv.values()], axis=1).sum(axis=1),
}).sort_index().loc[bt_start:bt_end]

REBALANCE_FREQ = 'W'
TOP_K = min(20, len(bt_assets))
LOOKBACK_DAYS = 126

def build_weights_from_predictions(pred_matrix: pd.DataFrame, *, pm_style: str) -> pd.DataFrame:
    rebal_dates = set(pd.Series(pred_matrix.index, index=pred_matrix.index).resample(REBALANCE_FREQ).last().dropna().tolist())
    w_last = pd.Series(0.0, index=bt_assets)
    rows = []
    for dt in pred_matrix.index:
        if dt in rebal_dates:
            row = pred_matrix.loc[dt].dropna().sort_values(ascending=False)
            top = row.head(TOP_K)
            candidates = [a for a, v in top.items() if np.isfinite(v) and float(v) > 0.0]
            if not candidates:
                w_last = pd.Series(0.0, index=bt_assets)
            else:
                if pm_style == '1N':
                    w_dict = equal_weight(candidates)
                elif pm_style == 'MPT':
                    w_dict = optimize_mpt(returns_matrix, candidates, dt, lookback_days=LOOKBACK_DAYS)
                else:
                    raise ValueError(f'Unknown pm_style: {pm_style!r}')
                w_last = pd.Series(0.0, index=bt_assets)
                for a, w in w_dict.items():
                    w_last[str(a)] = float(w)
        rows.append(w_last)
    return pd.DataFrame(rows, index=pred_matrix.index, columns=bt_assets).fillna(0.0)

cfg = BacktestConfig(initial_equity=1_000_000.0, transaction_cost_bps=5.0, mode='vectorized')

compare_rows = []
results = {}
for pm_style in ['1N', 'MPT']:
    w = build_weights_from_predictions(pred_matrix, pm_style=pm_style)
    res = run_backtest(close_prices, w, config=cfg)
    rpt = compute_backtest_report(result=res, close_prices=close_prices)
    results[pm_style] = (w, res, rpt)
    compare_rows.append({
        'style': pm_style,
        'Total Return [%]': float(rpt['Total Return [%]']),
        'CAGR [%]': float(rpt['CAGR [%]']),
        'Sharpe': float(rpt['Sharpe']),
        'Max Drawdown [%]': float(rpt['Max Drawdown [%]']),
    })
compare = pd.DataFrame(compare_rows).sort_values('Total Return [%]', ascending=False).reset_index(drop=True)
display(compare)

BASE_TITLE = 'Bayes Logistic Regression (VI)'
for pm_style in ['1N', 'MPT']:
    w, res, rpt = results[pm_style]
    title = BASE_TITLE + ' - ' + pm_style
    display(rpt.to_frame(title))
    layout = build_interactive_portfolio_layout(
        market_ohlcv=market_df,
        equity=res.equity,
        returns=res.returns,
        weights=res.weights,
        turnover=res.turnover,
        costs=res.costs,
        close_prices=close_prices,
        title=title,
    )
    show(layout)


Loading BokehJS ...

,style,Total Return [%],CAGR [%],Sharpe,Max Drawdown [%]
0,1N,20.272196,13.095023,0.813090,-17.262441
1,MPT,-0.798405,-0.532981,0.057541,-22.022878


,Bayes Logistic Regression (VI) - 1N
Start,2024-07-16 00:00:00
End,2026-01-16 00:00:00
Duration,549 days 00:00:00
Initial Equity,1000000.0
Final Equity,1202721.961048
Equity Peak,1232920.871974
Total Return [%],20.272196
CAGR [%],13.095023
Volatility (ann) [%],16.843844
Sharpe,0.81309


,Bayes Logistic Regression (VI) - MPT
Start,2024-07-16 00:00:00
End,2026-01-16 00:00:00
Duration,549 days 00:00:00
Initial Equity,1000000.0
Final Equity,992015.950532
Equity Peak,1043943.997646
Total Return [%],-0.798405
CAGR [%],-0.532981
Volatility (ann) [%],17.494073
Sharpe,0.057541
